In [1]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path.parent))
os.environ['VECTORIAN_CPP_IMPORT'] = "1"

In [2]:
import vectorian

In [ ]:
import spacy
#nlp = spacy.load("en_core_web_trf")
nlp = spacy.load("en_core_web_sm")

In [ ]:
import vectorian
from vectorian.embeddings import CompressedFastTextVectors, PretrainedGloVe

fasttext = CompressedFastTextVectors("../models/fasttext-en-mini.kv")
glove = PretrainedGloVe("6B", 50)

from vectorian.importers import NovelImporter
from vectorian.session import LabSession
from vectorian.embeddings import SpacyTransformerEmbedding

# contextual embeddings
#im = NovelImporter(nlp, embeddings=[SpacyTransformerEmbedding(nlp)])

im = NovelImporter(nlp)
doc = im("/Users/arbeit/Wise Children.txt")

# note: you may also specify StackedEmbedding here.
session = LabSession(
    [doc],
    [fasttext, glove],
    "default")

In [ ]:
session.interact(nlp)